In [1]:
import db

In [2]:
import coin_repository

In [3]:
from matplotlib import pyplot as plt
import pandas as pd
import math

In [4]:
import strategies as st

In [5]:
import simulator

In [6]:
sim = simulator.Simulator(st.GcdcStrategy(pd.DataFrame(coin_repository.CoinRepository('ETH').get_data())))

TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'

In [ ]:
# df = pd.DataFrame(coin_repository.CoinRepository('ETH').get_data())
# s = st.GcdcStrategy(df)

In [ ]:
# for x in df['close'].keys():
#     if s.has_to_buy(x):
#         print(x, "buy")
#     if s.has_to_sell(x):
#         print(x, "sell")

In [ ]:
# class MddCagr:
#     def __init__(self, coin_type):
#         self.coin_type = coin_type
#         self.coin_full_name = {
#             'ETH': 'ethereum',
#             'BTC': 'bitcoin',
#             'XRP': 'ripple'
#         }[coin_type]
    
#     def check_mddcagr(self):
#         (index, origin) = coin_repository.CoinRepository(f'{self.coin_full_name}').get_data_as_lists()
#         data = {}
#         new_index = index
#         close_series = pd.Series(origin['close'], new_index)
#         data['ma5'] = close_series.rolling(window=5).mean()
#         data['ma20'] = close_series.rolling(window=20).mean()
#         # df = pd.DataFrame(data, new_index)
#         # df.plot()
#         previous_ma5 = -1
#         previous_ma20 = -1
#         previous_close = -1
#         benefits = []
#         drawdowns = []
#         bought = False
#         max_close = -1

#         for tuple in zip(data['ma5'], data['ma20'], origin['close'], index):
#             if tuple[0] > tuple[1] and previous_ma5 <= previous_ma20:
#                 print(f"golden cross: {tuple}")
#                 bought = True
#                 if len(benefits) == 0:
#                     benefits.append(1)
#                 else:
#                     benefits.append(benefits[-1])
#             elif tuple[0] < tuple[1] and previous_ma5 >= previous_ma20:
#                 print(f"dead cross: {tuple}")
#                 bought = False
#                 benefits.append(benefits[-1] * (tuple[2] / previous_close))
#             else:
#                 if len(benefits) == 0:
#                     benefits.append(1)
#                 elif bought:
#                     benefits.append(benefits[-1] * (tuple[2] / previous_close))
#                 else:
#                     benefits.append(benefits[-1])

#         if (tuple[2] > max_close):
#             max_close = tuple[2]

#         drawdowns.append(1 - (tuple[2] / max_close))

#         previous_ma5 = tuple[0]
#         previous_ma20 = tuple[1]
#         previous_close = tuple[2]

#         df = pd.Series(benefits, index)
#         df.plot()

#         # df = pd.Series(drawdowns, index)
#         # df.plot()

#         print(f"{coin_full_name}MDD: {max(drawdowns) * 100}%")
#         print(f"CAGR: {(benefits[-1] - 1) * 100}%")